In [ ]:
!pip install --quiet timm pytorch-lightning==1.9.0 torchmetrics==0.11.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.8/825.8 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.2/517.2 kB 36.5 MB/s eta 0:00:00


In [ ]:
import os
import gc
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import timm
import pytorch_lightning as L

from typing import Dict, Any, List, Optional
from glob import glob
from tqdm.auto import tqdm
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold
from torchvision.io import read_image
from torchvision.transforms import v2 as transforms
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import train_test_split
from torchmetrics import Accuracy, F1Score
from google.colab import drive
from google.colab import drive
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, models, transforms
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from torch.utils.data import DataLoader

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r "/content/drive/MyDrive/이미지 데이터" /content/
data_dir = "/content/이미지 데이터"

## Data Preprocessing


In [ ]:
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import datasets, transforms

class CustomDataset(Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __len__(self):
        return len(self.subset)

    def __getitem__(self, idx):
        image, label = self.subset[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

# 데이터 전처리
transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform2 = transforms.Compose([
    transforms.Resize((336, 336)),
    transforms.RandomHorizontalFlip(p=0.5),  # 50% 확률로 좌우 반전
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # 랜덤 밝기, 대비, 채도 조정
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# ImageFolder로 데이터셋 로드
dataset = datasets.ImageFolder(root=data_dir)

# train, validation, test
train_size = int(0.7 * len(dataset))  # 70% for training
valid_size = int(0.15 * len(dataset))  # 15% for validation
test_size = len(dataset) - train_size - valid_size  # 15% for testing

train_dataset, valid_dataset, test_dataset = random_split(dataset, [train_size, valid_size, test_size])

# 각각의 데이터셋에 다른 transform을 적용
train_dataset1 = CustomDataset(train_dataset, transform=transform1)
train_dataset2 = CustomDataset(train_dataset, transform=transform2)
valid_dataset1 = CustomDataset(valid_dataset, transform=transform1)
valid_dataset2 = CustomDataset(valid_dataset, transform=transform2)
test_dataset1 = CustomDataset(test_dataset, transform=transform1)
test_dataset2 = CustomDataset(test_dataset, transform=transform2)

# DataLoader
batch_size = 32

train_loader1 = DataLoader(train_dataset1, batch_size=batch_size, shuffle=True, num_workers=12)
train_loader2 = DataLoader(train_dataset2, batch_size=batch_size, shuffle=True, num_workers=12)
valid_loader1 = DataLoader(valid_dataset1, batch_size=batch_size, shuffle=False, num_workers=12)
valid_loader2 = DataLoader(valid_dataset2, batch_size=batch_size, shuffle=False, num_workers=12)
test_loader1 = DataLoader(test_dataset1, batch_size=batch_size, shuffle=False, num_workers=12)
test_loader2 = DataLoader(test_dataset2, batch_size=batch_size, shuffle=False, num_workers=12)


## Eva02 Model

In [ ]:
# eva 모델 정의
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()
        self.model = timm.create_model("hf_hub:timm/eva02_small_patch14_336.mim_in22k_ft_in1k", pretrained=True)
        # self.model = timm.create_model("hf_hub:timm/eva_large_patch14_336.in22k_ft_in22k_in1k", pretrained=True)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))  # AdaptiveAvgPool2d
        self.clf = nn.Sequential(
            nn.SiLU(),
            nn.LazyLinear(1)
        )

    def forward(self, x, label=None):
        x = self.model.forward_features(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        logits = self.clf(x)

        loss = None
        if label is not None:
            loss = nn.BCEWithLogitsLoss()(logits.squeeze(-1), label.float())

        probs = torch.sigmoid(logits)
        return probs, loss


model = CustomModel().to('cuda')
criterion = nn.BCELoss()  # BCE Loss 사용
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=0.01)  # AdamW


## Resnet18 Model

In [ ]:
# resnet 모델
class ResNetBinaryClassifier(nn.Module):
    def __init__(self):
        super(ResNetBinaryClassifier, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, 1)

    def forward(self, x):
        x = self.model(x)
        return torch.sigmoid(x)

model = ResNetBinaryClassifier().to('cuda')
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## Train

In [ ]:
# train 정의
def train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=100):
    best_accuracy = 0.0
    best_model_path = 'best_model.pth'

    for epoch in range(num_epochs):
        train_loss = 0.0
        valid_loss = 0.0
        correct = 0
        total = 0

        # Training Loop
        model.train()
        for images, labels in train_loader:
            images, labels = images.to('cuda'), labels.to('cuda').float()

            optimizer.zero_grad()
            probs, loss = model(images, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

        # Validation Loop
        model.eval()
        with torch.no_grad():
            for images, labels in valid_loader:
                images, labels = images.to('cuda'), labels.to('cuda').float()

                probs, loss = model(images, labels)
                valid_loss += loss.item() * images.size(0)

                preds = (probs > 0.5).float()
                correct += (preds == labels.unsqueeze(1)).sum().item()
                total += labels.size(0)

        train_loss = train_loss / len(train_loader.dataset)
        valid_loss = valid_loss / len(valid_loader.dataset)
        accuracy = correct / total

        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Validation Loss: {valid_loss:.4f}, Accuracy: {accuracy:.4f}")

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with accuracy: {best_accuracy:.4f} at epoch {epoch+1}")

    print(f"Training complete. Best validation val_loss: {best_accuracy:.4f}")

def test_model(model, test_loader):
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()

    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to('cuda'), labels.to('cuda').float()

            probs, _ = model(images)
            preds = (probs > 0.5).float()

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    from sklearn.metrics import f1_score, accuracy_score
    f1 = f1_score(all_labels, all_preds)
    accuracy = accuracy_score(all_labels, all_preds)

    print(f"F1 Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")


In [ ]:
# 실행
train_model(model, train_loader, valid_loader, criterion, optimizer, num_epochs=100)

## evaluation

In [ ]:
# 모델 평가를 위한 가중치 로드
model.load_state_dict(torch.load('/content/resnet_best.pth'))
# model.load_state_dict(torch.load('/content/eva_best.pth'))
model.eval()

<ipython-input-70-ddd3342461db>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/resnet_best.pth'))


ResNetBinaryClassifier(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=Tr

## resnet

In [ ]:
test_loader1 = DataLoader(test_dataset1, batch_size=32, shuffle=False, num_workers=12)

model.load_state_dict(torch.load('/content/resnet_best.pth'))
model.eval()

all_preds = []
all_labels = []

# 테스트 루프
with torch.no_grad():
    for images, labels in test_loader1:
        images, labels = images.to('cuda'), labels.to('cuda').float()
        labels = labels.unsqueeze(1)

        outputs = model(images)
        preds = (outputs > 0.5).float()  # 예측값을 0.5 기준으로 이진화

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# F1 스코어와 정확도 계산 (이진 분류용)
f1 = f1_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, all_preds)

# 혼동 행렬 계산
cm = confusion_matrix(all_labels, all_preds)
print("resnet18")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{cm}")


<ipython-input-71-e2dc99668bbf>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/resnet_best.pth'))


resnet18
Accuracy: 0.8365
F1 Score: 0.8952
Confusion Matrix:
[[ 44  42]
 [ 10 222]]


## eva

In [ ]:

test_loader2 = DataLoader(test_dataset2, batch_size=32, shuffle=False, num_workers=12)

model.load_state_dict(torch.load('/content/eva_best.pth'))
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in test_loader2:
        images, labels = images.to('cuda'), labels.to('cuda').float()
        labels = labels.unsqueeze(1)

        outputs, _ = model(images)  # 예측값(outputs)과 손실(_)을 반환, 손실은 무시
        preds = (outputs > 0.5).float()  # 예측값을 0.5 기준으로 이진화

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# F1 스코어, 정확도 계산 (이진 분류용)
f1 = f1_score(all_labels, all_preds)
accuracy = accuracy_score(all_labels, all_preds)

# 혼동 행렬 계산
cm = confusion_matrix(all_labels, all_preds)

# 결과 출력
print("eva02")
print(f"Accuracy: {accuracy:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Confusion Matrix:\n{cm}")


<ipython-input-68-d663e9137ff3>:10: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/content/eva_best.pth'))


eva02
Accuracy: 0.8648
F1 Score: 0.9095
Confusion Matrix:
[[ 59  27]
 [ 16 216]]
